In [1]:
# NEW LOADING
import pandas as pd
import numpy as np
def load_csv(file):
    df = pd.read_csv(file)

    # NO PROJETO, CRIAR UM TRANSFORMER PARA ESSAS OPERAÇÕES!!
    # Converting the 'hearing_date' and 'ticket_issued_date' columns to datetime dtype.
    df['hearing_date'] = pd.to_datetime(df['hearing_date']) 
    df['ticket_issued_date'] = pd.to_datetime(df['ticket_issued_date'])

    # Two new columns displaying the emission of the ticket date and the hearing date month number 
    df['hearing_date_month'] = df.hearing_date.dt.month
    df['ticket_issued_date_month'] = df.ticket_issued_date.dt.month

    # A new column showing the time span from the emission of the ticket to the hearing date.
    df['hearing_wait'] = (df.hearing_date - df.ticket_issued_date).dt.days
    
    # Loading the addresses and geographical coordinates data.
    addresses = pd.read_csv('addresses.csv')
    latlons = pd.read_csv('latlons.csv')
    # Joining the DataFrames into a single one.
    coordinates = addresses.merge(latlons, on='address').drop('address', axis=1)

    # Inserting the dwellings coordinates.
    df = df.merge(coordinates, on='ticket_id')
    
    return df

train = load_csv('train.csv')

# Setting the columns of no use to the model, since they are not available in the 'test.csv'.
# This will prevent us to face Data Leakage.
to_drop = ['Unnamed: 0','payment_amount', 'payment_date','payment_status', 'balance_due', 'collection_status','compliance_detail']
train.drop(to_drop, axis=1, inplace=True)

# Dropping out the rows with NaN in the 'compliance' column.
train = train[pd.notna(train['compliance'])]
train.head(3)

/tmp/ipykernel_53658/3828651547.py:30: DtypeWarning: Columns (12,13,32) have mixed types.Specify dtype option on import or set low_memory=False.
  train = load_csv('train.csv')


,ticket_id,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,...,discount_amount,clean_up_cost,judgment_amount,grafitti_status,compliance,hearing_date_month,ticket_issued_date_month,hearing_wait,lat,lon
0,22056,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","INVESTMENT INC., MIDWEST MORTGAGE",2900.0,TYLER,NaN,3.0,S. WICKER,CHICAGO,...,0.0,0.0,305.0,NaN,0.0,3.0,3,369.0,42.390729,-83.124268
1,27586,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Michigan, Covenant House",4311.0,CENTRAL,NaN,2959.0,Martin Luther King,Detroit,...,0.0,0.0,855.0,NaN,1.0,5.0,4,378.0,42.326937,-83.135118
5,22046,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","KASIMU, UKWELI",6478.0,NORTHFIELD,NaN,2755.0,E. 17TH,LOG BEACH,...,0.0,0.0,305.0,NaN,0.0,3.0,5,323.0,42.145257,-83.208233


In [73]:
from sklearn.preprocessing import MinMaxScaler
def imputer_mean(df, cols):
    for col in cols:
        mean = df[col].mean()
        df[col].fillna(mean, inplace=True)
        #min_max(df, col)
    #scaler = MinMaxScaler()
    #scaler.fit_transform(df[cols])
    return df

num_cols = ['fine_amount', 'admin_fee', 'state_fee','late_fee', 'discount_amount',
            'clean_up_cost', 'judgment_amount', 'hearing_date_month',
            'ticket_issued_date_month', 'hearing_wait','lat', 'lon']
scaler = MinMaxScaler()
train_num = scaler.fit_transform(imputer_mean(train, num_cols)[num_cols])

In [78]:
a = pd.DataFrame({'A':[1,2,3, np.nan],
             'B':[np.nan, np.nan, 1, 20]})
min_max(a, 'B')

,A,B
0,1.0,NaN
1,2.0,NaN
2,3.0,0.0
3,NaN,1.0


In [79]:

imputer_mean(a, 'B')
a

,A,B
0,1.0,0.5
1,2.0,0.5
2,3.0,0.0
3,NaN,1.0


In [74]:
train.groubpy()

,ticket_id,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,...,discount_amount,clean_up_cost,judgment_amount,grafitti_status,compliance,hearing_date_month,ticket_issued_date_month,hearing_wait,lat,lon
0,22056,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","INVESTMENT INC., MIDWEST MORTGAGE",2900.0,TYLER,NaN,3.0,S. WICKER,CHICAGO,...,0.0,0.0,305.0,NaN,0.0,3.0,3,369.0,42.390729,-83.124268
1,27586,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Michigan, Covenant House",4311.0,CENTRAL,NaN,2959.0,Martin Luther King,Detroit,...,0.0,0.0,855.0,NaN,1.0,5.0,4,378.0,42.326937,-83.135118
5,22046,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","KASIMU, UKWELI",6478.0,NORTHFIELD,NaN,2755.0,E. 17TH,LOG BEACH,...,0.0,0.0,305.0,NaN,0.0,3.0,5,323.0,42.145257,-83.208233
6,18738,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Deerwood Development Group Inc, Deer",8027.0,BRENTWOOD,NaN,476.0,Garfield,Clinton,...,0.0,0.0,855.0,NaN,0.0,2.0,6,253.0,42.433466,-83.023493
7,18735,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Rafee Auto Services L.L.C., RAF",8228.0,MT ELLIOTT,NaN,8228.0,Mt. Elliott,Detroit,...,0.0,0.0,140.0,NaN,0.0,2.0,6,251.0,42.388641,-83.037858
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250278,285106,Department of Public Works,"Zizi, Josue","LUNDY, ANTONIO R",20009.0,NORTHLAWN,NaN,18715.0,MARLOWE,DETROIT,...,0.0,0.0,250.0,NaN,0.0,2.0,12,37.0,42.440228,-83.154829
250287,284650,Department of Public Works,"Lusk, Gertrina","HARDAWAY, WILLIE",15725.0,STEEL,NaN,NaN,P.O. BOX 27125,DETROIT,...,0.0,0.0,1130.0,NaN,0.0,1.0,12,10.0,42.406293,-83.171850
250288,285125,Department of Public Works,"Bell, Maydell","CHRISTIAN CHURCH, BRIGHT STAR",7152.0,CHICAGO,NaN,7152.0,W. CHICAGO,DETROIT,...,0.0,0.0,580.0,NaN,0.0,1.0,12,26.0,42.366529,-83.141897
250290,284881,Department of Public Works,"Johnson, Lois","INC., HANTZ GROUP",17403.0,MT ELLIOTT,NaN,1743.0,MT ELLIOTT,DETROIT,...,0.0,0.0,230.0,NaN,1.0,2.0,12,40.0,42.422081,-83.038656


In [59]:
def min_max(df, col):
    max_ = df[col].max()
    min_ = df[col].min()
    for i, element in zip(df.index, df[col]):
        df.loc[i, col] = (element - min_) / (max_ - min_)
    return df

In [61]:
a = pd.DataFrame({'A':[1,2,3, np.nan],
             'B':[np.nan, np.nan, 1, 20]})
min_max(a, 'B')

,A,B
0,1.0,NaN
1,2.0,NaN
2,3.0,0.0
3,NaN,1.0


In [4]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split


def load_csv(file):
    df = pd.read_csv(file, encoding='ISO-8859-1')

    # NO PROJETO, CRIAR UM TRANSFORMER PARA ESSAS OPERAÇÕES!!
    # Converting the 'hearing_date' and 'ticket_issued_date' columns to datetime dtype.
    df['hearing_date'] = pd.to_datetime(df['hearing_date']) 
    df['ticket_issued_date'] = pd.to_datetime(df['ticket_issued_date'])

    # Two new columns displaying the emission of the ticket date and the hearing date month number 
    df['hearing_date_month'] = df.hearing_date.dt.month
    df['ticket_issued_date_month'] = df.ticket_issued_date.dt.month

    # A new column showing the time span from the emission of the ticket to the hearing date.
    df['hearing_wait'] = (df.hearing_date - df.ticket_issued_date).dt.days
    
    # Loading the addresses and geographical coordinates data.
    addresses = pd.read_csv('addresses.csv')
    latlons = pd.read_csv('latlons.csv')
    # Joining the DataFrames into a single one.
    coordinates = addresses.merge(latlons, on='address').drop('address', axis=1)

    # Inserting the dwellings coordinates.
    df = df.merge(coordinates, on='ticket_id')
    
    return df

# Loading the training data.
train = load_csv('train.csv')

# Setting the columns of no use to the model, since they are not available in the 'test.csv'.
# This will prevent us to face Data Leakage.
to_drop = ['payment_amount', 'payment_date','payment_status', 'balance_due', 'collection_status','compliance_detail']
train.drop(to_drop, axis=1, inplace=True)

# Dropping out the rows with NaN in the 'compliance' column.
train = train[~pd.isnull(train['compliance'])]

# Filling the NaN values from the numerical columns with their mean.
def fillna_mean(df, cols):
    for col in cols:
        mean = df[col].mean()
        df[col].fillna(mean, inplace=True)
    return df

num_pipeline = Pipeline([
                ('normalizing', MinMaxScaler())                           
                        ])

# Treating text values.
class TextStats(TransformerMixin,BaseEstimator):
    
    def __init__(self, desired_columns):
        #print(desired_columns)
        self.desired_columns = desired_columns
    
    def fit(self, df):
        return self
    
    def transform(self,df):
        global train
        for column in train[self.desired_columns].columns:
            if column != 'compliance':
                # The number of compliance and non-compliance occurences according each category.
                cases = train.groupby([column, 'compliance']).size()

                # Number of occurences from each of the column´s categories
                total_cases = train.groupby([column]).size()

                # Filtering out the categories which have a low number of occurences from 'total_cases'.
                new_total_cases = total_cases.drop(total_cases[total_cases<total_cases.mean()].index, axis=0)

                # Measuring the overall disobedience levels for the categories available and turning it into a Series.
                non_compliance_ratios = cases.xs(0, level=1) / new_total_cases

                # Creating a name for the Series
                non_compliance_ratios.name = column + '_non_compliance_ratio'

                # The Series will become a new column in the DataFrame (in the project's context, 'new_train').
                # At the same time, we are dropping out the columns used (except for 'compliance'). Since their dtype is 'object', they are 
                # not going to be of use to most of the ML models available.
                df = df.join(non_compliance_ratios, on=column).drop(column, axis=1)
        # We are replacing the 'compliance' column for a clone named 'target' so that the binary digits occupy the last column of the
        # DataFrame.
        
        if 'compliance' in df.columns:
            df['target'] = df['compliance']
            df.drop('compliance', axis=1,inplace=True)
        
        # For the new column cells that received NaN as values, we'll assign them an arbitrary value (0.5)
        #imputer = SimpleImputer(fill_value=0.5, strategy='constant')
        #df = imputer.fit_transform(df.values)
        df.fillna(0.5, inplace=True)
        return df

def blight_model():
    global train
    # Text
    text_stats_cols = ['agency_name', 'inspector_name','violation_code', 'violation_description', 'disposition', 'compliance']
    train_text = TextStats(text_stats_cols).fit_transform(train[text_stats_cols])
    
    # Num
    num_cols = ['fine_amount', 'admin_fee', 'state_fee','late_fee', 'discount_amount',
            'clean_up_cost', 'judgment_amount', 'hearing_date_month',
            'ticket_issued_date_month', 'hearing_wait','lat', 'lon']
    train_nums = fillna_mean(train, num_cols)
    train_nums = num_pipeline.fit_transform(train[num_cols])
    
    train_finished = np.hstack((train_nums,train_text))
    X = train_finished[:, [i for i in range(17)]]
    y = train_finished[:, 17]
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
    
    clf = RandomForestClassifier(max_features=5, n_estimators= 50).fit(X_train, y_train)
    
    # Test set
    test = load_csv('test.csv')
    
    # Text
    l = ['agency_name', 'inspector_name','violation_code', 'violation_description', 'disposition']
    test_text = TextStats(text_stats_cols).fit_transform(test[l])
    
    # Num
    test_nums = fillna_mean(test, num_cols)
    test_nums = num_pipeline.fit_transform(test[num_cols])

    test_finished = np.hstack((test_nums, test_text))
    return pd.Series(clf.predict_proba(test_finished)[:, 1], index=test['ticket_id'].values)

blight_model()

/tmp/ipykernel_53658/2941420439.py:37: DtypeWarning: Columns (12,13,32) have mixed types.Specify dtype option on import or set low_memory=False.
  train = load_csv('train.csv')


284932    0.36
285362    0.10
285361    0.12
285338    0.26
285346    0.22
          ... 
376496    0.10
376497    0.10
376499    0.44
376500    0.44
369851    0.64
Length: 61001, dtype: float64